# Scrapping Tribunnews 

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs
from datetime import datetime
from dateutil import parser

In [2]:
data = {
    "keywords": "anies",
    "since_time": "2023-11-01",
    "until_time": "2023-11-10"
}

In [3]:
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
#     }
# url = f"https://www.tribunnews.com/index-news?date=2023-9-2&page="
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
# cek= soup.find('div', {"class": "paging"})
# links= cek.find_all('a')
# all_link=[]
# for link in links:
#         if link and 'href' in link.attrs:
#             link_href = link['href']
#             all_link.append(link_href)
# link=all_link[-1]
# parsed_url = urlparse(link)

# # Extract the "page" parameter value
# page_value = parse_qs(parsed_url.query).get('page', [])[0] if 'page' in parse_qs(parsed_url.query) else None

# print("Page value:", page_value)
# # body = soup.find('ul', {"class": "lsi"})
# # print(body)

In [4]:
def scrape_links(date, page_number):
    current_date = datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%-m-%-d")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }

    url = f"https://www.tribunnews.com/index-news?date={current_date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    body = soup.find('ul', {"class": "lsi"})
    articles = body.find_all('li', {"class": "ptb15"})
    
    links = []
    for article in articles:
        header = article.find('h3', {"class": "f16 fbo"})
        if header:
            link = header.find('a')
            if link and 'href' in link.attrs:
                link_href = link['href']
                links.append(link_href)
                
    print(f"Scraped {len(links)} links from page {page_number} url {url}")

    return links


In [5]:
def scrape_link_per_day(date, max_threads=5):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    current_date = datetime.strptime(date, "%Y-%m-%d").strftime("%Y-%-m-%-d")
    url = f"https://www.tribunnews.com/index-news?date={current_date}&page="
    response = requests.get(url + "1", headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    cek = soup.find('div', {"class": "paging"})
    links = cek.find_all('a')
    all_link = []
    for link in links:
        if link and 'href' in link.attrs:
            link_href = link['href']
            all_link.append(link_href)
    last_link = all_link[-1]

    # Extract the "page" parameter value from the last link
    parsed_url = urlparse(last_link)
    page_value = parse_qs(parsed_url.query).get('page', [])[0] if 'page' in parse_qs(parsed_url.query) else None
    print(page_value)
    # Check if page_value is an integer
    try:
        page_number = int(page_value)
    except (TypeError, ValueError):
        page_number = 1

    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        # Use list comprehension to submit tasks to the thread pool
        futures = [executor.submit(scrape_links, current_date, index) for index in range(1, page_number + 1)]

        # Collect results from completed futures
        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [6]:
link=scrape_link_per_day(data["since_time"])

38
Scraped 20 links from page 1 url https://www.tribunnews.com/index-news?date=2023-11-1&page=1
Scraped 20 links from page 2 url https://www.tribunnews.com/index-news?date=2023-11-1&page=2
Scraped 20 links from page 4 url https://www.tribunnews.com/index-news?date=2023-11-1&page=4
Scraped 20 links from page 6 url https://www.tribunnews.com/index-news?date=2023-11-1&page=6
Scraped 20 links from page 3 url https://www.tribunnews.com/index-news?date=2023-11-1&page=3
Scraped 20 links from page 5 url https://www.tribunnews.com/index-news?date=2023-11-1&page=5
Scraped 20 links from page 8 url https://www.tribunnews.com/index-news?date=2023-11-1&page=8
Scraped 20 links from page 7 url https://www.tribunnews.com/index-news?date=2023-11-1&page=7
Scraped 20 links from page 9 url https://www.tribunnews.com/index-news?date=2023-11-1&page=9
Scraped 20 links from page 11 url https://www.tribunnews.com/index-news?date=2023-11-1&page=11
Scraped 20 links from page 10 url https://www.tribunnews.com/inde

In [7]:
print(len(link))

760


In [8]:
results=[]

In [9]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('h1', {"id": "arttitle"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"   
                    # tanggal berita
                    date_elem = soup.find('time', {"class": "grey"})
                    if date_elem:
                        date_text = date_elem.text.strip()
                        date_part = ' '.join(date_text.split(',')[1:]).strip()
                        date_object = parser.parse(date_part)
                        formatted_date = date_object.strftime('%Y/%m/%d')
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"class": "side-article txt-article multi-fontsize"})
                    
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                            content_text += p.text.strip() + "\n"
                        
                        if content_text.strip():
                            content_text=content_text
                            content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                            content_text = ' '.join(content_text.split())
                        else:
                            content_text ="Content not found"
                    else:
                        content_text ="Content not found"

                    return{
                        'title': title_text,
                        'date': formatted_date,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [10]:
url = 'https://www.tribunnews.com/nasional/2023/11/01/wapres-maruf-amin-dan-panglima-tni-buka-gerakan-nasional-ketahanan-pangan-2023'
data_tribunnews=scrape_url(url)

In [11]:
print(data_tribunnews)

{'title': "Wapres Ma'ruf Amin dan Panglima TNI Buka Gerakan Nasional Ketahanan Pangan 2023", 'date': '2023/11/01', 'content': 'TRIBUNNEWS.COM, JAKARTA - Wakil Presiden RI Ma’ruf Amin dan Menteri Pertanian Amran Sulaiman menghadiri pembukaan Gerakan Nasional Ketahanan Pangan 2023 di Taman Pancasila, Cibitung, Kabupaten Bekasi, Rabu (1/11/2023).Gerakan yang diinisiasi oleh TNI ini dibuka oleh Wapres Ma’ruf Amin dan Panglima TNI Laksamana Yudo Margono.Selain Wapres dan Panglima, turut hadir pula Kapolri Jenderal Listyo Sigit dan kepala staf tiga matra, yakni Kepala Staf TNI AD (KSAD) Jenderal Agus Subiyanto, Kepala Staf TNI AL (KSAL) Laksamana Muhammad Ali, dan Kepala Staf TNI AU (KSAU) Marsekal Fadjar Prasetyo serta Menteri Pertanian Amran Sulaiman.Berdasarkan pantauan Tribunnews.com di lokasi, Wapres Ma’ruf Amin, Mentan Amran, Panglima TNI Yudo, dan Kapolri Listyo Sigit meninjau lokasi penggemukan sapi yang dikerjakan kelompok tani.Selain lokasi penggemukan sapi, para pejabat tinggi itu

In [13]:
for url in link:
    data_tribunnews = scrape_url(url)
    
    # Assuming "keywords" is a key in data_tribunnews
    keywords = ['prabowo', 'ganjar', 'gibran']

    # Check if any keyword is present in the lowercase version of 'title' or 'content'
    if any(keyword.lower() in data_tribunnews['title'].lower() or keyword.lower() in data_tribunnews.get('content', '').lower() for keyword in keywords):
        print("Data contains keywords:", data_tribunnews)
    else:
        print("News does not contain the specified keywords and will not be inserted into the database. URL:", data_tribunnews.get('link', 'Link not available'))

Data contains keywords: {'title': 'Ini Kata Arsjad Rasjid Menanggapi Kabar Sri Mulyani Gabung TPN Ganjar-Mahfud', 'date': '2023/11/01', 'content': 'Laporan Wartawan Tribunnews.com, Fersianus WakuTRIBUNNEWS.COM, JAKARTA - Ketua Tim Pemenangan Nasional (TPN) Ganjar Pranowo dan Mahfud MD, Arsjad Rasjid mengatakan kabar Sri Mulyani bergabung dalam timnya masih sebatas isu.Hal ini disampaikan Arsjad merespons pertanyaan awak media mengenai isu Menteri Keuangan itu akan gabung TPN Ganjar-Mahfud."Mungkin tadi Mbak sudah menjawab sendiri, itu masih isu ya," kata Arsjad dalam jumpa pers di Gedung High End, Jakarta, Rabu (1/11/2023).Arsjad juga mengatakan pihaknya membuka pintu bagi Menteri BUMN Erick Thohir jika ingin bergabung dalam TPN.Pernyataan ini disampaikan Arsjad terkait pertemuan Ketua Dewan Pakar TPN Ganjar-Mahfud, Sandiaga Uno dengan Erick beberapa waktu lalu."Kami kan selalu terbuka," ujarnya.Namun, Arsjad menyerahkan sepenuhnya kepada Erick apabila ingin bergabung dengan TPN Ganjar

KeyboardInterrupt: 